# Bayes Estimation

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')

## References

- https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/lecturenote05.html

In [22]:
df = pd.read_csv('/opt/datasetsRepo/smsspamcollection/SMSSpamCollection', sep='\t', names=['label', 'text'])

df['flag'] = df['label'].map({ "ham" : 0, "spam" : 1})

dataset = pd.concat([ 
    df.query('flag == 1').sample(50), 
    df.query('flag == 0').sample(50) 
], axis = 0).sample(frac=1)

dataset.head()

,label,text,flag
2541,ham,"They said if its gonna snow, it will start aro...",0
868,spam,PRIVATE! Your 2003 Account Statement for 07808...,1
1328,spam,Ur balance is now £500. Ur next question is: W...,1
2663,spam,Hello darling how are you today? I would love ...,1
4303,ham,Aiyar hard 2 type. U later free then tell me t...,0


## Naive Bayes

\begin{align}
    P(Y=y | X=x) &= \frac{P(X=x | Y=y) P(Y=y)}{P(X=x)}\\
    \\
    &\text{Where } \\
    P(X=x | Y=y) &= \prod_{\alpha=1}^{d} P([X]_\alpha = x_\alpha| Y = y)
\end{align}


- Naively assumes that all the features used are independently distrubuted variables given the label Y.
- for example given that there is an email where all the words are independent given the label spam/ham.

## Bayes Classifier

\begin{align*}
    h(\vec{x}) &= {argmax\atop{y}} \frac{P(\vec{x} | y) P(y)}{z}\\
    \\
    &= {argmax\atop{y}} P(y) \prod_{\alpha} P([\vec{X}]_\alpha | y)\\
    \\
    &= {argmax\atop{y}} ( log(P(y) + \sum_\alpha log P([\vec{X}]_\alpha | y))
\end{align*}


P.S. - In computer science we dont prefer multiplying probabilities due to muliple reasons(see reference section). Hence we take log and convert multiplication to addition.